### Import needed modules

In [1]:
import os
import numpy as np
import pandas as pd

from gurobipy import *

import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

### Create variables

In [3]:
m = Model('system')

arcs = [
    ("GRID", "ELEC"),
    ("GRID", "HP"),
    ("HP", "HEAT"),
    ("GAS", "CHP"),
    ("CHP", "ELEC"),
    ("CHP", "HEAT")
]

Academic license - for non-commercial use only


In [4]:
load_elec = pd.read_csv("load_elec.csv", index_col=0, squeeze=True)
load_heat = pd.read_csv("load_heat.csv", index_col=0, squeeze=True)

In [5]:
nb_of_periods = len(load_elec)

flows = m.addVars(arcs, nb_of_periods, name="flows")

### Heat pump

In [6]:
cop = 3.0

cstr_hp = m.addConstrs(
    (cop * flows["GRID", "HP", i] - flows["HP", "HEAT", i] == 0 
     for i in range(nb_of_periods)))

### Cogeneration

### Load (Elec + Heat)

### Optimize and plot results